# Basic CDMS Tutorial<a id='top' class="tocSkip"> </a>

## Preparing your environment

### Installing cdms2

```
conda create -n cdms -c conda-forge cdms
source activate cdms
```

### Bringing in some data


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Preparing-your-environment" data-toc-modified-id="Preparing-your-environment-0.1">Preparing your environment</a></span><ul class="toc-item"><li><span><a href="#Installing-cdms2" data-toc-modified-id="Installing-cdms2-0.1.1">Installing cdms2</a></span></li><li><span><a href="#Bringing-in-some-data" data-toc-modified-id="Bringing-in-some-data-0.1.2">Bringing in some data</a></span></li></ul></li><li><span><a href="#Opening-and-querying-a-file-for-reading" data-toc-modified-id="Opening-and-querying-a-file-for-reading-0.2">Opening and querying a file for reading</a></span></li><li><span><a href="#Querying-Variables-(in-file)" data-toc-modified-id="Querying-Variables-(in-file)-0.3">Querying Variables (in file)</a></span></li><li><span><a href="#Dimensions" data-toc-modified-id="Dimensions-0.4">Dimensions</a></span></li><li><span><a href="#Time-dimensions" data-toc-modified-id="Time-dimensions-0.5">Time dimensions</a></span></li></ul></li><li><span><a href="#Retrieving-data" data-toc-modified-id="Retrieving-data-1">Retrieving data</a></span></li><li><span><a href="#Manipulating-Data" data-toc-modified-id="Manipulating-Data-2">Manipulating Data</a></span></li><li><span><a href="#Creating-MV2-and-storing-them-in-files" data-toc-modified-id="Creating-MV2-and-storing-them-in-files-3">Creating MV2 and storing them in files</a></span></li><li><span><a href="#Saving-data" data-toc-modified-id="Saving-data-4">Saving data</a></span></li></ul></div>

In [ ]:
from __future__ import print_function
import cdat_info
import os, sys

data_path = cdat_info.get_sampledata_path()
cdat_info.download_sample_data_files(os.path.join(sys.prefix,"share","cdms2","test_data_files.txt"),data_path)

## Opening and querying a file for reading

In [ ]:
# Open a sample file
import cdms2

filename = os.path.join(data_path,"clt.nc")
f = cdms2.open(filename)

In [ ]:
# Query variables in the file
var = f.listvariable()
print("variables in the file:",var)

In [ ]:
# Query dimensions in the file
dims = f.listdimension()
print("Dimensions in the file:",dims)

In [ ]:
# Query file attributes
attr = f.listglobal()
print("File attributes:",attr)

## Querying Variables (in file)

You can further query the variables in the file without having to read them in memory

To create a `file variable` simply use square bracket: **[** and **]**

In [ ]:
clt = f["clt"]  # This is a file variable, not in memory

In [ ]:
# Print variable info to screen
clt.info()

In [ ]:
# Variable shape
sh = clt.shape
print("The variable shape is:",sh)

In [ ]:
# Variable id
name = clt.id
print("Variable id/name:",name)

In [ ]:
# The variable dimensions
axes = clt.getAxisList()
print("variable dimensions:",axes)

In [ ]:
# Variable attributes
attributes = clt.attributes
print("Variable attributes:",attributes.keys())

## Dimensions

In [ ]:
# Determine if an axis is time
for a in axes:
    if a.isTime():
        print("Axes %s is a time axis" % a.id)
    else:
        print("Axes %s is not a time axis" % a.id)

In [ ]:
# Similar functions exist for level, latitude and longitude
for a in axes:
    print(a.isLatitude(), a.isLongitude(), a.isLevel())

In [ ]:
# Similarly we can get one of these 4 types of dimension automatically
aTime = clt.getTime()
lat = clt.getLatitude()
lon = clt.getLongitude()

In [ ]:
# if such dimension does not exists None is returned
lev = clt.getLevel()
print("Level dim:",lev)

In [ ]:
# Any dimension can also by retrieved by its index
dim0 = clt.getAxis(0)
print("The first dim name is:",dim0.id)

In [ ]:
# Dimension information
dim0.info()

In [ ]:
# Accessing axis values
print("Latitude values:",clt.getLatitude()[:])

## Time dimensions

cdms is really good at dealing with times (see decdicated cdtime jupyter notebook for more on time)


In [ ]:
# Rather than raw (in file) values or indices it can be usefull to show/manipulate time 
# as 'component time'
tim = clt.getTime()
tc = tim.asComponentTime()
print("First 2 times are:",tc[:2])
# or 'relative times'
tr = tim.asRelativeTime("days since 2017")
print("first 2 times in days since 2017:", tr[:2])

# Retrieving data

In [ ]:
# Whole
clt =f("clt") # parentheis means read in memory
print("Shape:",clt.shape)

In [ ]:
# Partial, based on values in file
clt = f("clt",latitude=(0,90),longitude=(-180,180))
print("Shape:",clt.shape)

In [ ]:
# Based on indices
clt = f("clt",time=slice(0,12))
print("Shape:",clt.shape)

In [ ]:
# time can be retirieved based on actual dates (provided units are good in file)
clt = f("clt",time=("1980","1983-12-31"))
print("Shape:",clt.shape)

In [ ]:
# Data can also be read directly from a file variable
CLT = f["clt"]
clt = CLT(time=("1980","1984-12-31"),latitude=(0,90),longitude=slice(0,None))
print("Shape:",clt.shape)

In [ ]:
# Or from an exisitng variavle
clt2 = clt(time=slice(0,4))
print("Shape:",clt2.shape)

In [ ]:
# data can also be reordered based on dimensions
clt = f("clt",order="xty")
print("Shape:",clt.shape)

In [ ]:
# or use dimension indices
clt=f("clt", order="210")
print("Shape:",clt.shape)

In [ ]:
# or use dimension names
clt = f("clt",order="(longitude)(time)(latitude)")
print("Shape:",clt.shape)

# Manipulating Data

cdms variables are subclass of numpy, so for the most part anything you can do with numpy
can be done with cdms variables

In [ ]:
# Extract same month every years (from monthly data)
clt=f("clt")
subset = clt[::12]
print("Shape:",subset.shape)

In [ ]:
# cdms variable can be converted to raw numpy
nparray = clt.filled()
print(type(clt),type(nparray))

In [ ]:
# or masked arrays
maarray = clt.asma()
print(type(clt),type(maarray))

# Creating MV2 and storing them in files

In [ ]:
import MV2
# Create a cdms variable from a numpy (or numpy.ma) array
myvar = MV2.array(nparray)
myvar.id = "newclt"
myvar.info()

In [ ]:
# We can . add axes from other variables
myvar.setAxisList(clt.getAxisList())
myvar.info()

In [ ]:
# we can also add axes one at a time
for i in range(myvar.ndim):
    ax = clt.getAxis(i)
    print("Setting axis %i to %s" % (i,ax.id))
    myvar.setAxis(i,ax)
myvar.info()

In [ ]:
# We can also create axes manually
newtime = cdms2.createAxis(range(120))
newtime.id = "time" # name of dimension
newtime.designateTime()  # tell cdms to add attributes that make it time
newtime.units = "months since 2017"
myvar.setAxis(0,newtime)
myvar.info()  # Notice tikme changed

# Saving data

In [ ]:
 # By default cdms2 will save files in NetCDF4 compressed with no shuffle by defalted at level 1
print("Default Shuffle:",cdms2.getNetcdfShuffleFlag())
print("Default Deflate:",cdms2.getNetcdfDeflateFlag())
print("Default Deflate Level:",cdms2.getNetcdfDeflateLevelFlag())

In [ ]:
# Let's turn it all off so we get NetCDF3 classic files
value = 0
cdms2.setNetcdfShuffleFlag(value) ## where value is either 0 or 1
cdms2.setNetcdfDeflateFlag(value) ## where value is either 0 or 1
cdms2.setNetcdfDeflateLevelFlag(value) ## where value is a integer between 0 and 9 included
print("Shuffle:",cdms2.getNetcdfShuffleFlag())
print("Deflate:",cdms2.getNetcdfDeflateFlag())
print("Deflate Level:",cdms2.getNetcdfDeflateLevelFlag())

In [ ]:
# Let's open a file for writing
f2 = cdms2.open("mydata.nc","w") # "w" means open file for writing and erase if already here
f2.write(myvar)
f2.close()